In [ ]:
!pip install wandb -Uq

In [ ]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jaydencruz824 (jaydencruz824-belmont-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Expanded sweep config
sweep_config = {
    'method': 'random',   # options: grid, random, bayes
    'metric': {
        'name': 'test_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        # Learning rate search space
        'learning_rate': {
            'values': [0.1, 0.01, 0.001, 0.0005, 0.0001, 0.00005]
        },

        # Batch sizes
        'batch_size': {
            'values': [32, 64, 128, 256, 512]
        },

        # Optimizer
        'optimizer': {
            'values': ['sgd', 'adam', 'adamw', 'rmsprop']
        },

        # Weight decay (regularization)
        'weight_decay': {
            'values': [0.0, 1e-4, 1e-3, 1e-2]
        },

        # Dropout
        'dropout': {
            'values': [0.0, 0.2, 0.5]
        },

        # Activation function
        'activation': {
            'values': ['ReLU', 'LeakyReLU', 'GELU', 'SiLU', 'Tanh']
        },

        # Model type
        'model': {
            'values': ['MLP', 'CNN']
        },

        # For MLP only: number of layers and hidden size
        'num_layers': {
            'values': [1, 2, 3, 4]
        },
        'hidden_dim': {
            'values': [64, 128, 256, 512]
        }
    }
}

# Define available activation functions
activations = {
    "ReLU": torch.nn.ReLU(),
    "LeakyReLU": torch.nn.LeakyReLU(),
    "GELU": torch.nn.GELU(),
    "SiLU": torch.nn.SiLU(),
    "Tanh": torch.nn.Tanh()
}


# Sweep-compatible training function
def sweep_train():
    config = wandb.config

    # Pick activation
    activation = activations[config.activation]

    # Build model
    if config.model == 'MLP':
        model = MLP(dataset_info,
                    hidden_dim=config.hidden_dim,
                    num_layers=config.num_layers,
                    activation=activation,
                    dropout=config.dropout).to(device)
    else:
        model = CNN(dataset_info,
                    activation=activation,
                    dropout=config.dropout).to(device)

    # Choose optimizer
    if config.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=config.learning_rate,
                                    weight_decay=config.weight_decay)
    elif config.optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=config.learning_rate,
                                     weight_decay=config.weight_decay)
    elif config.optimizer == 'adamw':
        optimizer = torch.optim.AdamW(model.parameters(),
                                      lr=config.learning_rate,
                                      weight_decay=config.weight_decay)
    else:
        optimizer = torch.optim.RMSprop(model.parameters(),
                                        lr=config.learning_rate,
                                        weight_decay=config.weight_decay)

    # Recreate DataLoaders with sweep batch size
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=config.batch_size,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=config.batch_size,
                                              shuffle=False)

    # Train model
    results = train_model(model,
                          train_loader,
                          test_loader,
                          epochs=10,
                          lr=config.learning_rate,
                          optimizer=optimizer)

    return results


# Initialize and launch sweep
sweep_id = wandb.sweep(sweep_config, project="convnet_MNIST")
wandb.agent(sweep_id, function=sweep_train, count=10)  # run 10 experiments


NameError: name 'torch' is not defined